In [0]:
# !pip install pythainlp
# !pip install tltk
# !pip install emoji
# !pip install flask-ngrok

In [0]:
import pandas as pd
import numpy as np
import sklearn
import pythainlp
import matplotlib as plt
from flask_ngrok import run_with_ngrok
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Embedding, Conv1D,GlobalMaxPooling1D, CuDNNGRU, CuDNNLSTM, TimeDistributed, Flatten,Reshape,SpatialDropout1D, GRU, LSTM, Bidirectional,Dropout,BatchNormalization,MaxPooling1D
from keras.optimizers import Adam
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from keras.callbacks import ModelCheckpoint, EarlyStopping

from numpy import array
from numpy import asarray
from numpy import zeros
from pythainlp.spell import spell
from itertools import groupby
import tltk
from sklearn.utils import class_weight
# from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re
import matplotlib.pyplot as plt
import seaborn as sns
from pythainlp.tag.named_entity import ThaiNameTagger
from imblearn.metrics import classification_report_imbalanced
import tensorflow as tf
from keras import backend as K
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import AllKNN,RandomUnderSampler,ClusterCentroids
import pickle 
ner = ThaiNameTagger()
    
def inputData(filename):
   '''
    Should implement for your dataset
   '''
    file = open(filename,'r')
    data = np.array([e.split('|') for e in file.read().split('\n')])
    print('size of train '+str(data.shape))
    temp =[]
    for i in data :
            if len(i) == 2:
                temp.append([i[0],i[1]])

    temp
    data = temp
    del temp
    data[0]
    df = pd.DataFrame(data=data, columns=['input','output'])
    return preClean(df)

def preClean (df):
  '''
    Should implement for your dataset
  '''
    df = df[df.output != '']
    df = df[~df.input.str.contains('<->')]
    df = df.drop_duplicates(subset=(['input']),keep='first')
    s = df.groupby("output").input.agg(lambda x:len(x.unique()))
    ClassNum = np.concatenate((np.array(s.index.values).reshape((len(s.index.values),1)),
                    np.array(s.values).reshape((len(s.index.values),1))),axis=1)
    ClassNum = pd.DataFrame(ClassNum,columns=["output","count"])
    df = df[df["output"].isin(ClassNum[ClassNum["count"]>1000].output)]
    df = df[df["output"]!="No"]
    df = df[df["output"]!="Yes"]
    df = df[df["output"]!="nomatch"]
    return df

def tokenize(text): 
  '''
    tokenization
  '''
#     return word_tokenize(text,engine='newmm')
    return text.split()

def wordPreProcess(s):
  '''
    delete non alphabet except -?, empty string, and replace some entities with entity's name 
  '''
    out=[]
    th = [str(chr(a+ord('ก'))) for a in range(91)] 
    for w in s: 
        ss=''
        for i in range(len(w)):
            if w[i] in th or w[i].isalpha() or w[i] in '-?':
                ss+=w[i]
        if ss.strip() is not '' and len(ss.strip())>=2:
            if ner.get_ner(ss.strip())[0][2] in ['B-LOCATION', 'B-TIME', 'B-ORGANIZATION', 'B-DATE', 'B-PERSON']:
                out.append(ner.get_ner(ss.strip())[0][2])
            else:
                out.append(ss.strip())
    return out

def dataSplit(x,y,mode="Test"):
  '''
    split data 
    x is input feature
    y is output class
    mode = Train is used when you have no test dataset
    mode = Test is used when you have test dataset
  '''
    if mode == "Train":
        x_train, x_tmp, y_train, y_tmp = train_test_split(x, y, test_size=0.3, 
                                                        random_state=72,stratify = y)
        x_val, x_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size=0.3, random_state=72,stratify = y_tmp)
    elif mode == "Test":
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, 
                                                        random_state=72,stratify = y)
        x_test,y_test = np.array([]),np.array([])
    print("x_train shape:",x_train.shape)
    print("x_val shape:",x_val.shape)
    print("x_test shape:",x_test.shape)
    print("y_train shape:",y_train.shape)
    print("y_val shape:",y_val.shape)
    print("y_test shape:",y_test.shape)
    return ((x_train,y_train),(x_val,y_val),(x_test,y_test))

def imbDataSplit(x,y,mode="Under"):
  '''
    split data 
    x is input feature
    y is output class
    mode = Under is undersampling
    mode = Over is oversampling
    mode = Combine is both undersampling and oversampling
    defult split mode is Test
  '''
    if mode =="Combine":
        sme = SMOTETomek(random_state=42)
        x_res, y_res = sme.fit_resample(x, y)
    elif mode =="Under":
        allknn = AllKNN(random_state=42)
#     allknn = RandomUnderSampler(random_state=42)
        x_res, y_res = allknn.fit_resample(x, y)
    elif mode =="Over" :
#     sm = SMOTE(random_state=42)
        sm = RandomOverSampler(random_state=42)
        x_res, y_res = sm.fit_resample(x, y)
    return dataSplit(x_res,y_res,"Test") 


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

'''
Compatible with tensorflow backend
'''
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed


class Models:   
    
    def __init__(self, trainfile):
        train_df = inputData(trainfile)
        print("\nInput Data Completed\n")

        self.mmap = {k : v for k,v in zip(train_df["output"].unique(),range(len(train_df["output"].unique())))}
        self.rmap = {v : k for k,v in zip(train_df["output"].unique(),range(len(train_df["output"].unique())))}
        input_df,output_df = self.prepare(train_df)
        print("\nPrepare Completed\n")

        self.createDict(input_df)
        x = self.sentenceToVector(input_df)
        print("\nSentence to vector Completed\n")

        y = to_categorical(output_df)
        print("\nto categorical Completed\n")

        # ((x_train,y_train),(x_val,y_val),(x_test,y_test)) = imbDataSplit(x,y,"Under")
        # ((x_train,y_train),(x_val,y_val),_) = imbDataSplit(x,y)
        ((x_train,y_train),(x_val,y_val),_) = dataSplit(x,y)
        print("\nSplit data Completed\n")

#         input_all = pd.concat([input_df,test_input])
        self.createEmbeddingMatrix(input_df,"Word2Vec")
        print("\nCreate embedding matrix Completed\n")

        self.classNum(output_df)
        self.classWeight(output_df)
        print("\nclassnum&classweight Completed\n")

        self.model = self.GRUnn()
        self.model.fit(x=x_train, y=y_train, batch_size=8, verbose=1, epochs=3, \
                        class_weight=self.class_weights, 
                        validation_data=(x_val,y_val))
        print("\nTrain Completed\n")

    def evaluate(self,testfile):
        test_df = inputData(testfile)
        print("\nInput Data Completed\n")
        
        test_input,test_output = self.prepare(test_df)
        print("\nPrepare Completed\n")
        
        x_test = self.sentenceToVector(test_input)
        print("\nSentence to vector Completed\n")
        
        y_test = to_categorical(test_output)
        print("\nto categorical Completed\n")
        
        self.EvalByClass(x_test,y_test)
        self.Eval(x_test,y_test)
        print("\nEvaluate Completed\n")
        
    def predict(self,sentence):
        x_df = self.textPreProcess(pd.DataFrame([sentence],columns=['input']))
        x = self.sentenceToVector(x_df)
        y = self.model.predict(x)
#         y_pred = np.argmax(y,axis=1)
        y_df = pd.DataFrame([[np.argmax(y),np.max(y)*100]],columns=['output','confidence'])
        return self.categoricalToNumber(y_df,Reverse = True).values[0],np.max(y)*100

    def createDict(self,train_df):
      '''
        Create custom dict to index from dataset
      '''
        self.max_sentence_len = 0
        self.word2index = {"keras_mask_zero" : 0}
        self.index2word = {0 : "keras_mask_zero"}


        for s in train_df.values:
            self.max_sentence_len = max(len(s),self.max_sentence_len)
            for w in s:
                if w not in self.word2index:
                    self.word2index[w] = len(self.word2index)
                    self.index2word[len(self.index2word)] = w
        self.word2index["UNK"] = len(self.word2index)
        self.index2word[len(self.index2word)] = "UNK"
        self.vocab_size = len(self.word2index)
        print("vocab size :",self.vocab_size)
        print("max sentence length :",self.max_sentence_len)
        
    def textPreProcess(self,df):
        '''
        Pre-Process text from dataset
        '''
        input_df = df['input']       
        input_df = input_df.apply(tokenize)
        input_df = input_df.apply(wordPreProcess)
        return input_df


    def prepare(self,df):
        '''
        Pre-Process all dataset
        '''      
        return self.textPreProcess(df),self.categoricalToNumber(df)

    def sentenceToVector(self,df):
        '''
        tranform sentence to vector
        ''' 
        X = []
        for s in df.values:
            X.append(s)

        x = []
        for sentence in X:
            tmp = []
            for w in sentence:
                if w in self.word2index:
                    tmp.append(self.word2index[w])
                else:
                    tmp.append(self.word2index["UNK"])
            while(len(tmp) < self.max_sentence_len):
                tmp.append(0)
            x.append(tmp)
        return np.array(x)
    def categoricalToNumber(self,df,Reverse = False):
        '''
        switch between class name and class index
        Reverse = False for class name to class index
        Reverse = True for class index to class name
        ''' 
        if not Reverse :
            return df["output"].map(self.mmap)
        else:
            return df["output"].map(self.rmap)
        return df
    def createEmbeddingMatrix(self,input_df,mode):
        '''
        create embedding matrix 
        mode = Word2Vec, FastText 
        ''' 
        self.WV_size = 300
        if mode == "Word2Vec":
            self.model_embed = Word2Vec(input_df.values, size =self.WV_size , window=10, min_count=1, workers=10)
            self.model_embed.train(input_df.values, total_examples=len(input_df.values), epochs=10, compute_loss=True)
        elif mode == "FastText":
            self.model_embed = FastText(size=self.WV_size, window=10, min_count=1, sentences=input_df.values, iter=10)

        self.embedding_matrix = zeros((len(self.model_embed.wv.vocab)+1, self.WV_size))
        i = 1
        for word in self.model_embed.wv.vocab :
            embedding_vector = self.model_embed[word]
            if embedding_vector is not None:
                self.embedding_matrix[i] = embedding_vector
            i+=1

    def classNum(self,output_df):
        '''
        count number of class
        ''' 
        self.classnum = len(output_df.unique())
        print("Class Number: ",self.classnum)

    def classWeight(self,output_df):
        '''
        compute class weight from dataset
        ''' 
        self.class_weights = class_weight.compute_class_weight('balanced',np.unique(output_df),output_df)
        plt.bar([x for x in range(len(self.class_weights))], self.class_weights)
        print(self.class_weights)

    def FeedForword(self):
        '''
        FeedForward model
        ''' 
        inp = Input(shape=(self.max_sentence_len,))
        x = Embedding(len(self.model_embed.wv.vocab)+1, self.WV_size, weights = [self.embedding_matrix], 
                      input_length=self.max_sentence_len, trainable=True)(inp)
        x = Flatten()(x)
        x = Dense(200,activation = 'relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dropout(0.4)(x)
        x = Dense(self.classnum, activation = 'softmax')(x)
        model = Model(inputs = inp , outputs = x)
        model.compile(Adam(), loss=[focal_loss(alpha=.25, gamma=2)], metrics=['acc'])
        model.summary()
        return model

    def Conv(self):
        '''
        Convolutional Model
        ''' 
        inp = Input(shape=(self.max_sentence_len,))
        x = Embedding(len(self.model_embed.wv.vocab)+1, self.WV_size, weights = [self.embedding_matrix], 
                      input_length=self.max_sentence_len, trainable=True)(inp)

        x = SpatialDropout1D(0.4)(x)
        x = Conv1D(8,5,strides=1,activation='relu')(x)
        x = GlobalMaxPooling1D()(x)
        x = Dense(200,activation = 'relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dropout(0.4)(x)
        x = Dense(self.classnum, activation = 'softmax')(x)
        model = Model(inputs = inp , outputs = x)
        model.compile(Adam(), loss='categorical_crossentropy', metrics=['acc'])
        model.summary()
        return model

    def GRUnn(self):
        '''
        Recurrent Model
        '''
        inp = Input(shape=(self.max_sentence_len,))
        x = Embedding(len(self.model_embed.wv.vocab)+1, self.WV_size, weights = [self.embedding_matrix], 
                      input_length=self.max_sentence_len, mask_zero=False, trainable=True)(inp)
    #     x = CuDNNGRU(self.WV_size,return_sequences=False, return_state=False)(x)
        x = Bidirectional(CuDNNGRU(self.WV_size,return_sequences=False, return_state=False))(x)
    #     x = CuDNNLSTM(self.WV_size,return_sequences=False, return_state=False)(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dense(50,activation = 'relu')(x)
        x = Dense(200,activation = 'relu')(x)
        x = Dense(50,activation = 'relu')(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dense(50,activation = 'relu')(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dense(self.classnum, activation = 'softmax')(x)
        model = Model(inputs = inp , outputs = x)
        model.compile(Adam(), loss='categorical_crossentropy', metrics=['acc'])
        model.summary()
        return model

    def Conv_GRU_nn(self):
        '''
        Convolutional and Recurrent Model
        '''
        inp = Input(shape=(self.max_sentence_len,))
        x = Embedding(len(self.model_embed.wv.vocab)+1, self.WV_size, weights = [self.embedding_matrix], 
                      input_length=self.max_sentence_len, trainable=True)(inp)
        x = SpatialDropout1D(0.2)(x)
        x = Conv1D(8,5,strides=1,padding='valid',activation='relu')(x)
        x = TimeDistributed(Dense(5, activation = 'relu'))(x)
        x = Flatten()(x)
        x = Dense(self.max_sentence_len*self.WV_size,activation = 'relu')(x)

        x = Reshape((self.max_sentence_len, self.WV_size))(x)


        x = Bidirectional(CuDNNGRU(self.WV_size,return_sequences=False, return_state=False))(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dense(50,activation = 'relu')(x)
        x = Dense(200,activation = 'relu')(x)
        x = Dense(50,activation = 'relu')(x)
        x = Dense(100,activation = 'relu')(x)
        x = Dense(self.classnum, activation = 'softmax')(x)
        model = Model(inputs = inp , outputs = x)
        model.compile(Adam(), loss='categorical_crossentropy', metrics=['acc'])
        model.summary()
        return model
        
    def Eval(self,x_test,y_test):
        y_pred = self.model.predict(x_test)
        y_pred = np.argmax(y_pred,axis=1)
        y_real = np.argmax(y_test,axis=1)

        print("f1_score  : ",f1_score(y_real,y_pred,average="macro"))
        print("accuracy  : ",accuracy_score(y_real,y_pred))
        print("precision : ",precision_score(y_real,y_pred,average="macro"))
        print("recall    : ",recall_score(y_real,y_pred,average="macro"))

    def EvalByClass(self,x_test,y_test):
        y_pred = self,model.predict(x_test)
        y_pred = np.argmax(y_pred,axis=1)
        y_real = np.argmax(y_test,axis=1)
        y_pred_c = [list() for i in range(classnum)]
        y_real_c = [list() for i in range(classnum)]
        for i in range(len(y_pred)):
                y_pred_c[y_real[i]].append(y_pred[i])
                y_real_c[y_real[i]].append(y_real[i])
        y_pred_c = np.array(y_pred_c)
        y_real_c = np.array(y_real_c)
        print("0","acc","f1","precision","recall")
        score =[]
        for i in range(classnum):
            if y_pred_c[i] != []:
                score.append([accuracy_score(y_pred_c[i],y_real_c[i]),
                f1_score(y_pred_c[i],y_real_c[i],average='micro'),
                precision_score(y_pred_c[i],y_real_c[i],average='micro'),
                recall_score(y_pred_c[i],y_real_c[i],average='micro')])
        score = np.array(score)
        print("all",score.mean(axis=0))
        print(classification_report_imbalanced(y_real, y_pred))
    def save(self,filename):
        with open(filename+".model", 'wb') as modelfile:
            pickle.dump(self, modelfile)
    def load(filename):
        with open(filename+".model", 'rb') as modelfile:
            tmp = pickle.load(modelfile)
        return tmp

In [0]:
# path = modelPath 

In [0]:
from flask import Flask
app = Flask(__name__)

@app.route('/predict/<sentence>')
def predict(sentence):
    global path
    K.clear_session()
    model = Models.load(path)
    y = model.predict(sentence)
    K.clear_session()
    return "action : " + y[0] + "<br>confidence : " + str(y[1])

run_with_ngrok(app)
if __name__  == '__main__':
  app.run()